# AudioDataModule

In [5]:
import importlib
importlib.reload(importlib.import_module("src.dataModule"))
from src.dataModule import AudioDataModule
from transformers import AutoTokenizer
from transformers import WhisperTokenizer
from prep.processor import get_tokenizer

dm = AudioDataModule()
dm.setup("fit")
dm.setup("validate")
dm.setup("test")

# Batch

In [6]:
from tqdm import tqdm
import time

start_time = time.time()
for batch in tqdm(dm.train_dataloader()):
    x, y = batch
    pass

print("--- %s seconds ---" % (time.time() - start_time))

  2%|▏         | 32/1421 [00:11<08:10,  2.83it/s]


KeyboardInterrupt: 

---
# Test

In [8]:
from datasets import load_dataset
dataset = load_dataset("google/fleurs", "he_il", split="test", cache_dir='datasets/test', trust_remote_code=True)
dataset = dataset.rename_column("transcription", "normalized_text")

In [2]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-100h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-100h")

Some weights of the model checkpoint at facebook/wav2vec2-base-100h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.mask_time_emb_vector']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-100h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav

In [3]:
audio_sample = dataset[2]
inputs = processor(audio_sample["audio"]["array"], sampling_rate=audio_sample["audio"]["sampling_rate"], return_tensors="pt")

In [4]:
x = inputs.input_values

with torch.no_grad():
  logits = model(x).logits

  

In [2]:
from processor import get_tokenizer
from charactertokenizer import CharacterTokenizer
from string import whitespace, punctuation
from hebrew.chars import HEBREW_CHARS
from transformers import WhisperFeatureExtractor
# tokenizer = get_tokenizer()

vocab = [hebrew_char.char for hebrew_char in HEBREW_CHARS] + list(whitespace) + list(punctuation)
model_max_length = 2048
tokenizer = CharacterTokenizer(vocab, model_max_length)

example = "!הפירמידה הגדולה של גיזה היא היחידה מבין שבעת הפלאים שעדיין קיימת היום"

tokens = tokenizer(example)["input_ids"]
print(tokens)

print(tokenizer.decode(tokens, skip_special_tokens=True))

ModuleNotFoundError: No module named 'processor'

In [24]:
def tokenization(examples, tokenizer):
    return tokenizer(examples["normalized_text"], padding="max_length", return_tensors="pt")


dataset = dataset.map(tokenization, fn_kwargs={"tokenizer": tokenizer}, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/792 [00:00<?, ? examples/s]

In [25]:
for i in range(100):
    print(dataset[i]["input_ids"].shape)

torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([2048])
torch.Size([